In [32]:
import requests
from typing import Optional, List, Dict, Any
from langchain.llms.base import LLM

class LocalLLM(LLM):
    # ประกาศฟิลด์ให้ Pydantic รู้จัก
    api_url: str
    model_name: str

    @property
    def _llm_type(self) -> str:
        """ชื่อชนิดของ LLM เพื่อ logging ภายใน LangChain"""
        return "local_llm"

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        return {"model": self.model_name, "api_url": self.api_url}

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> str:
        """เรียก API และคืน response มาเป็นสตริง"""
        res = requests.post(
            f"{self.api_url}/api/generate",
            json={
                "model": self.model_name,
                "prompt": prompt,
                "stream": False,
            },
        )
        res.raise_for_status()
        return res.json().get("response", "")

API_URL = "http://localhost:11434"
MODEL_NAME = "scb10x/typhoon2.1-gemma3-4b:latest"
llm = LocalLLM(api_url=API_URL, model_name=MODEL_NAME)


In [61]:
# ใช้งาน LLM ผ่าน DeepInfra (OpenAI compatible API)
from openai import OpenAI
from typing import Optional, List, Dict, Any
from langchain.llms.base import LLM
from pydantic import PrivateAttr

class DeepInfraLLM(LLM):
    api_key: str
    base_url: str
    model_name: str
    _client: OpenAI = PrivateAttr()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._client = OpenAI(api_key=self.api_key, base_url=self.base_url)

    @property
    def _llm_type(self) -> str:
        return "deepinfra_llm"

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        return {"model": self.model_name, "base_url": self.base_url}

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> str:
        messages = [{"role": "user", "content": prompt}]
        response = self._client.chat.completions.create(
            model=self.model_name,
            messages=messages,
        )
        return response.choices[0].message.content

# ตั้งค่า DeepInfra
DEEPINFRA_API_KEY = "iaLJrht7O6tDk1KOn6lyE20RUD6Nof7U"
DEEPINFRA_BASE_URL = "https://api.deepinfra.com/v1/openai"
DEEPINFRA_MODEL = "google/gemma-3-27b-it"

llm = DeepInfraLLM(
    api_key=DEEPINFRA_API_KEY,
    base_url=DEEPINFRA_BASE_URL,
    model_name=DEEPINFRA_MODEL
)

In [16]:
from langchain.embeddings import HuggingFaceEmbeddings
# Wrap SentenceTransformer ผ่าน LangChain
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

In [17]:
from langchain.document_loaders import TextLoader
import glob

paths = glob.glob("Raw-data-from-TTT\**\*.txt", recursive=True)
documents = []
for p in paths:
    loader = TextLoader(p, encoding="utf-8")
    documents.extend(loader.load())


In [18]:
from langchain.vectorstores import FAISS
# สร้างและเก็บ Document embeddings ลงใน FAISS index
vectorstore = FAISS.from_documents(documents, embeddings)
# บันทึก index ไว้ใช้งานครั้งถัดไป
vectorstore.save_local("faiss_index")

In [64]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# Custom prompt template
custom_template = """
บทสนทนาที่ผ่านมา:
{chat_history}

ใช้บริบทต่อไปนี้ในการตอบคำถามที่อยู่ท้ายบท
ฉันคือ TTT-Assistant ผู้ช่วย AI ของบริษัท TTT Brothers Co., Ltd.ค่ะ
หากฉันไม่ทราบคำตอบ ให้ตอบเพียงว่าขอโทษนะคะฉันไม่ทราบค่ะ จะไม่พยายามแต่งคำตอบขึ้นมา

ข้อมูลที่มี:
{context}

คำถาม: {question}

คำตอบ: """

CUSTOM_PROMPT = PromptTemplate(
    template=custom_template,
    input_variables=["chat_history", "context", "question"]
)

# สร้าง Memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# สร้าง QA Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": CUSTOM_PROMPT}
)

In [67]:
# คำถามตัวอย่าง
question = "มีใครเป็นพนักงานดีเด่นบ้าง"
# เรียก chain พร้อมส่งคำถาม
result = qa_chain({"question": question})

# ดูประวัติการสนทนา
#print("Chat history:\n", memory.load_memory_variables({}))

print(result.get("answer"))

มีค่ะ พนักงานดีเด่นประจำปี 2567 คือ:

อันดับ 1: คุณอิศรา ดาราสาย (Software Developer)
อันดับ 2: คุณภุณดึ วายพา (Executive Secretary)
อันดับ 3: คุณเดชชนินทร์ ลำพาย (Software Developer)

